In [1]:
sys.path.append('../')
from util.util import load_ckp
from model.model import BertForMultiLable

In [2]:
import torch
bert_checkpoint_path = "../results/best_model/best_model_bert_ts1000.pt"
model = BertForMultiLable(170)
model = torch.nn.DataParallel(model)
# load check point
checkpoint = torch.load(bert_checkpoint_path)
# initialize state_dict from checkpoint to model
model.load_state_dict(checkpoint['state_dict'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [3]:
from torch.utils.data import DataLoader
from data_structure.question import BERTQuestionDataset
from transformers import BertTokenizer
from sklearn import metrics
from sklearn import preprocessing
import pandas as pd
vocab_vocab_path = "../../data/tags/20211110/ts1000/_2_1_commonTags.csv"
tag_vocab = pd.read_csv(vocab_vocab_path)
tag_list = tag_vocab["tag"].astype(str).tolist()
mlb = preprocessing.MultiLabelBinarizer()
mlb.fit([tag_list])
input_test = "../../data/questions/Valid_Questions54TS1000.pkl"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
test = pd.read_pickle(input_test)
# hyperparameters
TEST_BATCH_SIZE = 64
test_set = BERTQuestionDataset(test, mlb, tokenizer)

test_data_loader = DataLoader(test_set,
                                batch_size=TEST_BATCH_SIZE,
                                num_workers=0
                                )

In [4]:
device = torch.device(
        'cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

DataParallel(
  (module): BertForMultiLable(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [5]:

model.eval()
fin_targets = []
fin_outputs = []
with torch.no_grad():
    for batch_idx, data in enumerate(test_data_loader, 0):
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(
            device, dtype=torch.long)

        targets = data['labels'].to(device, dtype=torch.float)
        outputs = model(
            input_ids=ids, token_type_ids=token_type_ids, attention_mask=mask)

        fin_targets.extend(targets.cpu().detach().numpy().tolist())
        fin_outputs.extend(torch.sigmoid(
            outputs).cpu().detach().numpy().tolist())
    print("load data complete")

In [12]:
print("load data complete")
test(fin_outputs, fin_targets)

load data complete


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy Score = 0.3438165167890438
F1 Score (Micro) = 0.6520124645769318
F1 Score (Macro) = 0.33852220682401435
Recall Score (Micro) = 0.5389461780781347
Recall Score (Macro) = 0.2974287666015974
Precision Score (Micro) = 0.8251143873919674
Precision Score (Macro) = 0.4936575438766366


In [11]:
import numpy as np
def test(outputs, targets):
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    recall_score_micro = metrics.recall_score(
        targets, outputs, average='micro')
    recall_score_macro = metrics.recall_score(
        targets, outputs, average='macro')
    precision_score_micro = metrics.precision_score(
        targets, outputs, average='micro')
    precision_score_macro = metrics.precision_score(
        targets, outputs, average='macro')
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    print(f"Recall Score (Micro) = {recall_score_micro}")
    print(f"Recall Score (Macro) = {recall_score_macro}")
    print(f"Precision Score (Micro) = {precision_score_micro}")
    print(f"Precision Score (Macro) = {precision_score_macro}")